In [ ]:
import logging
import warnings
import nest_asyncio
from datasets import Dataset
import pandas as pd
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    answer_similarity,
    answer_correctness,
    context_recall,
    context_precision,
)
#from ragas.langchain import RagasEvaluatorChain
# from ragas.langchain.evalchain import RagasEvaluatorChain
from test_chains import kwf_chain
from paths import get_paths
from config import LlmParam
#ChatParam
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
import InstructorEmbedding

In [ ]:
LLAMA_FILE = "llama-2-7b-chat.Q5_K_M.gguf"
MISTRAL_FILE = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"

paths_dir = get_paths(model_file=LLAMA_FILE, model_name="LLAMA2")
mistral_paths = get_paths(model_file=MISTRAL_FILE, model_name="MISTRAL")
PROJECT_DIR = paths_dir["PROJECT_DIR"]
DATA_DIR = paths_dir["DATA_DIR"]
LLAMA_DIR = paths_dir["MODEL_DIR"]
MISTRAL_DIR = mistral_paths["MODEL_DIR"]
CHROMA_VECTOR_DIR = paths_dir["CHROMA_VECTOR_DIR"]
PREPARED_CHUNK_DIR = paths_dir["PREPARED_CHUNK_DIR"]
VECTOR_STORE = paths_dir["VECTOR_STORE"]
TEMPERATURE = LlmParam.TEMPERATURE
N_GPU_LAYERS = LlmParam.N_GPU_LAYERS
N_TOKENS = LlmParam.N_TOKENS
KWF_TEMPLATE = LlmParam.KWF_TEMPLATE

In [ ]:
base_eval = '../data/BaseEvaluations.xlsx'
eval_df = pd.read_excel(base_eval)

In [ ]:
eval_df = eval_df.head(3)

In [ ]:
eval_df["Question expert"] = eval_df["Question expert"].astype(str)
eval_df["Réponse expert"] = eval_df["Réponse expert"].astype(str)
eval_df["Passage"] = eval_df["Passage"].astype(str)

In [ ]:
questions_experte = eval_df["Question expert"] + eval_df["Mots clés Question Expert"]
reponses_experte = eval_df["Réponse expert"]
context = eval_df["Passage"]

In [ ]:
embedding = HuggingFaceInstructEmbeddings(
        model_name="hkunlp/instructor-large", model_kwargs={"device": "cuda"}
    )

In [ ]:
answers = []
contexts = []

In [ ]:
i = 0
for question in questions_experte:
	i = i+1
	if i <=10:
		print(f"Traitement de la question {i}: {question} ...")
		#question = str(question)
		keyword_part = question.split("?")[-1]
		logging.info(f"Mots imposés: {keyword_part} ...")
		memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="question",
    return_messages=True,
    output_key="answer")
		prompt = PromptTemplate(
									input_variables=["context", "chat_history", "question"],
									template=KWF_TEMPLATE,
					)
		chain = kwf_chain(
						keyword_part, prompt, memory, embedding,
						PREPARED_CHUNK_DIR,
		)
		input_structure = {"question": question}

		response = chain.invoke(input_structure)
		answers.append(response["answer"])

		#exit()
		#chunk["source_documents"][i].page_content
		contexts.append([context.page_content for context in response["source_documents"]])

In [ ]:
df = pd.DataFrame({"quetions":questions_experte, "reponses": answers, "ground_truth": reponses_experte, "context": contexts})

In [ ]:
df.to_csv("questions_experte_response.csv", index=False)

# Méthodes Statistiques 
## Métriques basées sur la comparaison des mots ou suite de  mots (n-grams)
- Score `BLEU`
- Score `ROUGE` 
- Score `METEOR`

In [ ]:
import evaluate

In [ ]:
reference = "Le soleil brille dans le ciel bleu"

In [ ]:
prediction = "L'été est ma saison préférée car il fait chaud et ensoleillé"

In [ ]:
bleu = evaluate.load('bleu')
scores_bleu = bleu.compute(predictions=[prediction], references=[reference])

In [ ]:
scores_bleu

| Score  | Réference          | Prédiction |
| :--------------- |:---------------:| -----:|
|1  |   Le soleil brille dans le ciel bleu       |  Le soleil brille dans le ciel bleu |
| 0  | Le soleil brille dans le ciel bleu             |   Le ciel est couvert et il pleut |
|  0 | Le soleil brille dans le ciel bleu         |    L'été est ma saison préférée car il fait chaud et ensoleillé |

In [ ]:
rouge = evaluate.load('rouge')
scores_rouge = rouge.compute(predictions=[prediction], references=[prediction])

In [ ]:
scores_rouge

In [ ]:
pred1 = "Le ciel est couvert et il pleut"
pred2 = "L'été est ma saison préférée car il fait chaud et ensoleillé"

In [ ]:
scores_rouge1 = rouge.compute(predictions=[pred1], references=[reference])
scores_rouge2 = rouge.compute(predictions=[pred2], references=[reference])

In [ ]:
scores_rouge1

In [ ]:
scores_rouge2

| Score  | Réference          | Prédiction |
| :--------------- |:---------------:| -----:|
|rouge1 1, rouge2 1, rougeL 1  |   Le soleil brille dans le ciel bleu       |  Le soleil brille dans le ciel bleu |
| rouge1 0.28, rouge2 0.16, rougeL 0.28  | Le soleil brille dans le ciel bleu             |   Le ciel est couvert et il pleut |
|  rouge1 0, rouge2 0, rougeL 0 | Le soleil brille dans le ciel bleu         |    L'été est ma saison préférée car il fait chaud et ensoleillé |

In [ ]:

wer = evaluate.load("wer")

wer_score = wer.compute(predictions=[prediction], references=[reference])


In [ ]:
wer_score

In [ ]:
reference = "Le chat est sur le tapis"
pred1 = "Le chat repose sur le tapis"

In [ ]:
meteor = evaluate.load("meteor")

In [ ]:
meteor_score = meteor.compute(predictions=[pred1], references=[reference])

In [ ]:
meteor_score

In [ ]:
p1 = "Le soleil brille dans le ciel bleu"
p2 = "L'été est ma saison préférée car il fait chaud et ensoleillé"
p3 =  "Le ciel est couvert et il pleut"

In [ ]:
meteor_score = meteor.compute(predictions=[p3], references=[p1])

In [ ]:
meteor_score 

| Score  | Réference          | Prédiction |
| :--------------- |:---------------:| -----:|
|0.99  |   Le chat est sur le tapis       |  Le chat est sur le tapis |
| 0.8  | Le chat est sur le tapis             |   Le chat repose sur le tapis |
|  0 | Le soleil brille dans le ciel bleu         |    L'été est ma saison préférée car il fait chaud et ensoleillé |
|  0 | Le soleil brille dans le ciel bleu         |    Le ciel est couvert et il pleut|

## 

## Métriques basées sur des modèles d'embedding
- `BertScore`

In [ ]:
bert = evaluate.load("bertscore")

In [ ]:
p1_1 = "Les chats dorment dans le jardin"
p1_2 = "Les chats se reposent dans le jardin"

In [ ]:
bert_score = bert.compute(predictions=[p1_1], references=[p1_1], lang="fr") #rescale_with_baseline=True) #model="microsoft/deberta-xlarge-mnli")

In [ ]:
bert_score

| Score avec l'option `rescale_with_baseline=False`  | Réference          | Prédiction |
| :--------------- |:---------------:| -----:|
|1  |   Les chats dorment dans le jardin       |  Les chats dorment dans le jardin |
| 0.87 | Le chat est sur le tapis             |   Les chats se reposent dans le jardin |
|  0.78 | Il faut se conformer au RGPD         |    On est obligé d'être en règle par rapport au RGPD|
|  0.71 | ll faut se conformer au RGPD          |   Les chats se reposent dans le jardin |
|  0.69 | ll faut se conformer au RGPD          |    Le Gernica a été peint par Picasso|

In [ ]:
reference = "Le Gernica a été peint par Picasso" #Il est important d'être conforme au RGPD" #Il faut se conformer au réglement général de protection des données" #"Une sanction d'un million d'euros peut s'appliquer en cas de non conformité avec le RG"
#reference = "Il faut se conformer au RGPD" #Une sanction d'un million d'euros peut s'appliquer en cas de non conformité avec le RG"
predictions = "On est obligé d'être en règle par rapport au RGPD"

In [ ]:
ref1 = "Les chats dorment dans le jardin"
ref2 = "Le chat est sur le tapis"
ref3 = "Il faut se conformer au RGPD"
pred1  = "Les chats se reposent dans le jardin"
pred2 = "On est obligé d'être en règle par rapport au RGPD"
pred3 = "Les chats se reposent dans le jardin"
pred4 = "Le Gernica a été peint par Picasso"

In [ ]:
b1 = bert.compute(predictions=[pred4], references=[ref3], lang="fr", rescale_with_baseline=True)

In [ ]:
b1


| Score avec l'option `rescale_with_baseline=True`  | Réference          | Prédiction |
| :--------------- |:---------------:| -----:|
|1  |   Les chats dorment dans le jardin       |  Les chats dorment dans le jardin |
| 0.44 | Le chat est sur le tapis             |   Les chats se reposent dans le jardin |
|  0.43 | Il faut se conformer au RGPD         |    On est obligé d'être en règle par rapport au RGPD|
|  0.23 | ll faut se conformer au RGPD          |   Les chats se reposent dans le jardin |
|  0.17 | ll faut se conformer au RGPD          |    Le Gernica a été peint par Picasso|

In [ ]:
predictions = ["hello there", "general kenob"]
references = ["hello there", "general kenobi"]
bleurt = evaluate.load("bleurt", module_type="metric")
results = bleurt.compute(predictions=predictions, references=references)

## Evaluation sur les réponses du RAG

In [ ]:
import pandas as pd

In [ ]:
test_eval = pd.read_csv("questions_experte_response.csv")

In [ ]:
test_eval.head()

In [ ]:
metriques = {"bleu": evaluate.load('bleu'),
             'rouge': evaluate.load('rouge'), 
             'meteor': evaluate.load('meteor'),
             'bert': evaluate.load('bertscore')}

In [ ]:
references  = test_eval['ground_truth'].values
predictions = test_eval['reponses'].values

In [ ]:
bleuScore = []
rougeScore = []
meteorScore = []
bertScore = []



In [ ]:
references = test_eval["ground_truth"]
predictions = test_eval["reponses"]

In [ ]:
bleuscore = []
rougescore = []
meteorscore = []
bertscore = []
for i in range(len(references)):
 bleu = metriques["bleu"].compute(references=[references[i]], predictions=[predictions[i]])
 rouge = metriques["rouge"].compute(references=[references[i]], predictions=[predictions[i]])
 meteor = metriques["meteor"].compute(references=[references[i]], predictions=[predictions[i]])
 bert = metriques["bert"].compute(references=[references[i]], predictions=[predictions[i]], lang="fr")
 bleuscore.append(bleu["bleu"])
 rougescore.append({"rouge1": rouge["rouge1"], "rouge2" : rouge["rouge2"], "rougeL": rouge["rougeL"]})
 meteorscore.append(meteor["meteor"])
 bertscore.append(bert["precision"][0])
 #print(bert)

In [ ]:
bleuscore

In [ ]:
rougescore

In [ ]:
meteorscore

In [ ]:
bertscore

In [ ]:
test_eval["bleuScore"] = bleuscore
test_eval["rougeScore"] = rougescore
test_eval["meteorScore"] = meteorscore
test_eval["bertScore"] = bertscore

In [ ]:
test_eval.to_excel("EvaluationStatisquesEmbedding.xlsx" , index=False)

## LLM Evaluation

- FrameWork DeepEval un equivalent de pytest pour les LLMs 
- GEval Evaluation par LLM: Evaluation par gpt par défaut 

In [ ]:
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.metrics import GEval

In [ ]:
input = "Les administrations de plus de cinquante employés sont obligées de publier en ligne non seulement les documents administratifs communiqués selon le CRPA, mais aussi ceux inclus dans le Répertoire des Informations Publiques (RIP), ainsi que les bases de données et les données régulièrement mises à jour qui ont un intérêt économique, social, sanitaire ou environnemental"

In [ ]:
output = "Les articles L. 312-1-1 et L. 312-1-3 du CRPA imposent aux administrations de mettre en ligne des documents administratifs sur leur site internet, dans les délais prévus par la législation. Ces obligations s'appliquent aux administrations de plus de cinquante employés, ainsi qu'aux collectivités territoriales et aux établissements publics de coopération intercommunale. Les documents administratifs doivent être mis en ligne dans un délai de trente jours à compter de la date de leur création, sauf dans les cas où une mise en ligne plus rapide est nécessaire pour répondre à une demande expresse de communication. Les administrations doivent également mettre en ligne les documents qui leur sont demandés, même s'ils n'ont pas encore été créés, dans un délai de vingt jours à compter de la date de la demande. Enfin, les administrations doivent mettre en ligne les documents qui leur sont communiqués par les autres administrations, dans un délai de vingt jours à compter de la date de la communication. Les obligations de mise en ligne sont prévues par les articles L. 312-1-1 et L. 312-1-3 du CRPA, et sont applicables aux administrations de plus de cinquante employés, ainsi qu'aux collectivités territoriales et aux établissements publics de coopération intercommunale. Dispose d’un menu contextuel"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from deepeval.models.base_model import DeepEvalBaseLLM

class LLAMA2(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        device = "cuda" # the device to load the model onto

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)[0]

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "LLAMA 2"




In [ ]:
!huggingface-cli login --token $HUGGINGFACE_TOKEN

In [ ]:
#hf_model = "meta-llama/Llama-2-70b-chat" #mistralai/Mistral-7B-v0.1" #"meta-llama/Llama-2-70b-chat" 
#hf_model = "mistralai/Mistral-7B-v0.1"
hf_model = "meta-llama/Llama-2-7b"

model = AutoModelForCausalLM.from_pretrained(hf_model)
tokenizer = AutoTokenizer.from_pretrained(hf_model)

llama2 = LLAMA2(model=model, tokenizer=tokenizer)


In [ ]:
test_case = LLMTestCase(input=input, actual_output=output)
coherence_metric = GEval(
    name="Coherence",
    criteria="Coherence - la qualité collective de toutes les phrases dans la sortie acutuelle",
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    model=llama2
)

coherence_metric.measure(test_case)
print(coherence_metric.score)
print(coherence_metric.reason)